In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 자연처리 프레임워크 및 라이브러리 설치하기

In [2]:
# colab에 Mecab설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [3]:
%cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [4]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 61.9 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-02 07:52:55--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::34cc:ea4a, 2406:da00:ff00::22c2:513, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=K1XyoRFadih0xpxFiKN4i0zpW0c%3D&E

In [5]:
# 필요 프레임워크 및 라이브러리 준비
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Cleaned Dataset

/content/drive/MyDrive/Colab Notebooks/NLP_Classification/sentiment/Cleaned Dataset


# 데이터 로드

In [7]:
total_data_10X = pd.read_table('clean_total_data(app&cosmetic&movie&shopping&game)_10X.txt')
print(len(total_data_10X))
total_data_10X.head()

595286


,document,label
0,잘먹고 있어요 또 구입할게요,1
1,정말 접착력은 최고입니다,1
2,이건 좀 제 입맛에는 별로였어요,0
3,아니 왜 실행안되요,0
4,여러 사람이 동시에 접속하여 의사소통 할 수 있는 점이 가장 마음에 든다 특히 해외...,1


# 토큰화

In [8]:
# 불용어 정의 (한국어 조사, 접속사 등) 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [9]:
mecab=Mecab()
X_total_10X = []
for sentence in total_data_10X['document']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_total_10X.append(temp_X)
print(len(total_data_10X))
total_data_10X.head()

595286


,document,label
0,잘먹고 있어요 또 구입할게요,1
1,정말 접착력은 최고입니다,1
2,이건 좀 제 입맛에는 별로였어요,0
3,아니 왜 실행안되요,0
4,여러 사람이 동시에 접속하여 의사소통 할 수 있는 점이 가장 마음에 든다 특히 해외...,1


# 정수 인코딩

In [10]:
vocab_size = 9237
tokenizer_10X = Tokenizer()
tokenizer_10X.fit_on_texts(X_total_10X)
tokenizer_10X = Tokenizer(num_words = vocab_size +2, oov_token ='OOV')
tokenizer_10X.fit_on_texts(X_total_10X)
X_total_10X = tokenizer_10X.texts_to_sequences(X_total_10X)
y_total_10X = np.array(total_data_10X['label'])

# 빈 샘플(empty samples)제거

In [11]:
drop_total_10X = [index for index, sentence in enumerate(X_total_10X) if len(sentence) < 1]
X_total_10X = np.delete(X_total_10X, drop_total_10X, axis=0)
y_total_10X = np.delete(y_total_10X, drop_total_10X, axis=0)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


# 패딩

In [12]:
max_len = 45
X_total_10X = pad_sequences(X_total_10X, maxlen = max_len)

#  모델 훈련

In [13]:
# 필요한 프레임워크 및 라이브러리 임포트
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [16]:
# 필요한 매트릭 선언, AUC-ROC, F1-Score, AUC-PR 에 필요한 지표 모두 체크
import tensorflow as tf

ROCauc = tf.keras.metrics.AUC(curve="ROC") # auc_1
PRauc = tf.keras.metrics.AUC(curve="PR") # auc_2
RECALL = tf.keras.metrics.Recall()
PRECISION = tf.keras.metrics.Precision()
TP = tf.keras.metrics.TruePositives()
TN = tf.keras.metrics.TrueNegatives()
FP = tf.keras.metrics.FalsePositives()
FN = tf.keras.metrics.FalseNegatives()

In [14]:
embedding_dim = 128
# dropout_prob = (0.5, 0.8)
num_filters = 128

In [50]:
dropout_prob1 = (0.8, 0.5)
dropout_prob2 = (0.2, 0.5)
dropout_prob3 = (0.5, 0.2)
dropout_prob4 = (0.0, 0.0)
dropout_prob5 = (0.1, 0.3)
dropout_prob6 = (0.1, 0.1)

In [27]:
model_input = Input(shape = (max_len,))
z1 = Embedding(vocab_size+2, embedding_dim, input_length = max_len, name="embedding")(model_input)
z1 = Dropout(dropout_prob1[0])(z1)

In [28]:
conv_blocks = []
for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z1)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

In [29]:
z1 = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z1 = Dropout(dropout_prob1[1])(z1)
z1 = Dense(128, activation="relu")(z1)
model_output = Dense(1, activation="sigmoid")(z1)
model_z1 = Model(model_input, model_output)

In [36]:
# dropout_prob2
model_input = Input(shape = (max_len,))
z2 = Embedding(vocab_size+2, embedding_dim, input_length = max_len, name="embedding")(model_input)
z2 = Dropout(dropout_prob2[0])(z2)

conv_blocks = []
for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z2)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

z2 = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z2 = Dropout(dropout_prob2[1])(z2)
z2 = Dense(128, activation="relu")(z2)
model_output = Dense(1, activation="sigmoid")(z2)
model_z2 = Model(model_input, model_output)

In [39]:
# dropout_prob3
model_input = Input(shape = (max_len,))
z3 = Embedding(vocab_size+2, embedding_dim, input_length = max_len, name="embedding")(model_input)
z3 = Dropout(dropout_prob3[0])(z3)

conv_blocks = []
for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z3)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

z3 = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z3 = Dropout(dropout_prob3[1])(z3)
z3 = Dense(128, activation="relu")(z3)
model_output = Dense(1, activation="sigmoid")(z3)
model_z3 = Model(model_input, model_output)

In [44]:
# dropout_prob4
model_input = Input(shape = (max_len,))
z4 = Embedding(vocab_size+2, embedding_dim, input_length = max_len, name="embedding")(model_input)
z4 = Dropout(dropout_prob4[0])(z4)

conv_blocks = []
for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z4)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

z4 = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z4 = Dropout(dropout_prob4[1])(z4)
z4 = Dense(128, activation="relu")(z4)
model_output = Dense(1, activation="sigmoid")(z4)
model_z4 = Model(model_input, model_output)

In [45]:
# dropout_prob5
model_input = Input(shape = (max_len,))
z5 = Embedding(vocab_size+2, embedding_dim, input_length = max_len, name="embedding")(model_input)
z5 = Dropout(dropout_prob5[0])(z5)

conv_blocks = []
for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z5)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

z5 = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z5 = Dropout(dropout_prob5[1])(z5)
z5 = Dense(128, activation="relu")(z5)
model_output = Dense(1, activation="sigmoid")(z5)
model_z5 = Model(model_input, model_output)

In [51]:
# dropout_prob6
model_input = Input(shape = (max_len,))
z6 = Embedding(vocab_size+2, embedding_dim, input_length = max_len, name="embedding")(model_input)
z6 = Dropout(dropout_prob6[0])(z6)

conv_blocks = []
for sz in [3, 4, 5]:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z6)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

z6 = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z6 = Dropout(dropout_prob6[1])(z6)
z6 = Dense(128, activation="relu")(z6)
model_output = Dense(1, activation="sigmoid")(z6)
model_z6 = Model(model_input, model_output)

In [32]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)

In [33]:
model_z1.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc', ROCauc, PRauc, TP, TN, FP, FN])
mc_z1 = ModelCheckpoint('10_CNN_1D_KoNLPy(Mecab)_Total((movie&shopping&game&target10X)_v0.1.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model_z1.fit(X_total_10X, y_total_10X, epochs=30, callbacks=[es, mc_z1], batch_size=100, validation_split=0.2)

Epoch 1/30
4763/4763 [==============================] - 117s 21ms/step - loss: 0.4075 - acc: 0.8116 - auc: 0.8954 - auc_1: 0.9061 - true_positives: 211166.0000 - true_negatives: 175329.0000 - false_positives: 45595.0000 - false_negatives: 44128.0000 - val_loss: 0.3562 - val_acc: 0.8502 - val_auc: 0.9284 - val_auc_1: 0.9365 - val_true_positives: 55797.0000 - val_true_negatives: 45428.0000 - val_false_positives: 10015.0000 - val_false_negatives: 7815.0000

Epoch 00001: val_acc improved from -inf to 0.85024, saving model to 10_CNN_1D_KoNLPy(Mecab)_Total((movie&shopping&game&target10X)_v0.1.h5
Epoch 2/30
4763/4763 [==============================] - 100s 21ms/step - loss: 0.3568 - acc: 0.8420 - auc: 0.9210 - auc_1: 0.9297 - true_positives: 217256.0000 - true_negatives: 183724.0000 - false_positives: 37200.0000 - false_negatives: 38038.0000 - val_loss: 0.3445 - val_acc: 0.8574 - val_auc: 0.9341 - val_auc_1: 0.9413 - val_true_positives: 55851.0000 - val_true_negatives: 46232.0000 - val_false_

In [37]:
model_z2.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc', ROCauc, PRauc, TP, TN, FP, FN])
mc_z2 = ModelCheckpoint('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.2.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model_z2.fit(X_total_10X, y_total_10X, epochs=30, callbacks=[es, mc_z2], batch_size=100, validation_split=0.2)

Epoch 1/30
4763/4763 [==============================] - 102s 21ms/step - loss: 0.3478 - acc: 0.8459 - auc: 0.9276 - auc_1: 0.9357 - true_positives: 274300.0000 - true_negatives: 231761.0000 - false_positives: 44606.0000 - false_negatives: 44606.0000 - val_loss: 0.3007 - val_acc: 0.8714 - val_auc: 0.9460 - val_auc_1: 0.9530 - val_true_positives: 54909.0000 - val_true_negatives: 48839.0000 - val_false_positives: 6604.0000 - val_false_negatives: 8703.0000

Epoch 00001: val_acc improved from -inf to 0.87143, saving model to 10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.2.h5
Epoch 2/30
4763/4763 [==============================] - 100s 21ms/step - loss: 0.2859 - acc: 0.8784 - auc: 0.9498 - auc_1: 0.9564 - true_positives: 225584.0000 - true_negatives: 192710.0000 - false_positives: 28214.0000 - false_negatives: 29710.0000 - val_loss: 0.2828 - val_acc: 0.8785 - val_auc: 0.9510 - val_auc_1: 0.9572 - val_true_positives: 56521.0000 - val_true_negatives: 48067.0000 - val_false_

In [40]:
model_z3.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc', ROCauc, PRauc, TP, TN, FP, FN])
mc_z3 = ModelCheckpoint('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.3.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model_z3.fit(X_total_10X, y_total_10X, epochs=30, callbacks=[es, mc_z3], batch_size=100, validation_split=0.2)

Epoch 1/30
4763/4763 [==============================] - 103s 21ms/step - loss: 0.3548 - acc: 0.8414 - auc: 0.9278 - auc_1: 0.9362 - true_positives: 273204.0000 - true_negatives: 232217.0000 - false_positives: 44150.0000 - false_negatives: 45702.0000 - val_loss: 0.3006 - val_acc: 0.8707 - val_auc: 0.9452 - val_auc_1: 0.9523 - val_true_positives: 56133.0000 - val_true_negatives: 47528.0000 - val_false_positives: 7915.0000 - val_false_negatives: 7479.0000

Epoch 00001: val_acc improved from -inf to 0.87070, saving model to 10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.3.h5
Epoch 2/30
4763/4763 [==============================] - 103s 22ms/step - loss: 0.2979 - acc: 0.8714 - auc: 0.9454 - auc_1: 0.9526 - true_positives: 223631.0000 - true_negatives: 191323.0000 - false_positives: 29601.0000 - false_negatives: 31663.0000 - val_loss: 0.2895 - val_acc: 0.8760 - val_auc: 0.9492 - val_auc_1: 0.9563 - val_true_positives: 56874.0000 - val_true_negatives: 47423.0000 - val_false_

In [46]:
model_z4.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc', ROCauc, PRauc, TP, TN, FP, FN])
mc_z4 = ModelCheckpoint('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.4.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model_z4.fit(X_total_10X, y_total_10X, epochs=30, callbacks=[es, mc_z4], batch_size=100, validation_split=0.2)

Epoch 1/30
4763/4763 [==============================] - 98s 20ms/step - loss: 0.3192 - acc: 0.8610 - auc: 0.9402 - auc_1: 0.9477 - true_positives: 278114.0000 - true_negatives: 236981.0000 - false_positives: 39386.0000 - false_negatives: 40792.0000 - val_loss: 0.2804 - val_acc: 0.8811 - val_auc: 0.9520 - val_auc_1: 0.9581 - val_true_positives: 56976.0000 - val_true_negatives: 47929.0000 - val_false_positives: 7514.0000 - val_false_negatives: 6636.0000

Epoch 00001: val_acc improved from -inf to 0.88115, saving model to 10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.4.h5
Epoch 2/30
4763/4763 [==============================] - 100s 21ms/step - loss: 0.2407 - acc: 0.9004 - auc: 0.9644 - auc_1: 0.9688 - true_positives: 231352.0000 - true_negatives: 197458.0000 - false_positives: 23466.0000 - false_negatives: 23942.0000 - val_loss: 0.2745 - val_acc: 0.8837 - val_auc: 0.9545 - val_auc_1: 0.9599 - val_true_positives: 56753.0000 - val_true_negatives: 48455.0000 - val_false_p

In [48]:
model_z5.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc', ROCauc, PRauc, TP, TN, FP, FN])
mc_z5 = ModelCheckpoint('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.5.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model_z5.fit(X_total_10X, y_total_10X, epochs=30, callbacks=[es, mc_z5], batch_size=100, validation_split=0.2)

Epoch 1/30
4763/4763 [==============================] - 99s 20ms/step - loss: 0.3350 - acc: 0.8519 - auc: 0.9280 - auc_1: 0.9320 - true_positives: 275778.0000 - true_negatives: 233575.0000 - false_positives: 42792.0000 - false_negatives: 43128.0000 - val_loss: 0.2911 - val_acc: 0.8760 - val_auc: 0.9486 - val_auc_1: 0.9553 - val_true_positives: 55330.0000 - val_true_negatives: 48968.0000 - val_false_positives: 6475.0000 - val_false_negatives: 8282.0000

Epoch 00001: val_acc improved from -inf to 0.87605, saving model to 10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.5.h5
Epoch 2/30
4763/4763 [==============================] - 96s 20ms/step - loss: 0.2664 - acc: 0.8873 - auc: 0.9564 - auc_1: 0.9622 - true_positives: 227900.0000 - true_negatives: 194642.0000 - false_positives: 26282.0000 - false_negatives: 27394.0000 - val_loss: 0.2776 - val_acc: 0.8815 - val_auc: 0.9530 - val_auc_1: 0.9590 - val_true_positives: 56803.0000 - val_true_negatives: 48146.0000 - val_false_po

In [52]:
model_z6.compile(loss="binary_crossentropy", optimizer="adam", metrics=['acc', ROCauc, PRauc, TP, TN, FP, FN])
mc_z6 = ModelCheckpoint('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.6.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
history = model_z6.fit(X_total_10X, y_total_10X, epochs=30, callbacks=[es, mc_z6], batch_size=100, validation_split=0.2)

Epoch 1/30
4763/4763 [==============================] - 98s 20ms/step - loss: 0.3276 - acc: 0.8564 - auc: 0.9355 - auc_1: 0.9422 - true_positives: 277421.0000 - true_negatives: 235115.0000 - false_positives: 41252.0000 - false_negatives: 41485.0000 - val_loss: 0.2905 - val_acc: 0.8771 - val_auc: 0.9503 - val_auc_1: 0.9565 - val_true_positives: 54557.0000 - val_true_negatives: 49866.0000 - val_false_positives: 5577.0000 - val_false_negatives: 9055.0000

Epoch 00001: val_acc improved from -inf to 0.87710, saving model to 10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.6.h5
Epoch 2/30
4763/4763 [==============================] - 99s 21ms/step - loss: 0.2560 - acc: 0.8921 - auc: 0.9598 - auc_1: 0.9650 - true_positives: 229260.0000 - true_negatives: 195575.0000 - false_positives: 25349.0000 - false_negatives: 26034.0000 - val_loss: 0.2745 - val_acc: 0.8840 - val_auc: 0.9542 - val_auc_1: 0.9603 - val_true_positives: 57263.0000 - val_true_negatives: 47985.0000 - val_false_po

# 모델 성능평가(타겟데이터 기반)

In [34]:
# 평가용 토탈 데이터셋2(화장품+앱)

# 데이터 로드
total_data2 = pd.read_table('clean_total_data(app&cosmetic).txt')

# 토크나이징
X_test_total2 = []
for sentence in total_data2['document']:
    temp_X2 = mecab.morphs(sentence)
    temp_X2 = [word for word in temp_X2 if not word in stopwords]
    X_test_total2.append(temp_X2)

# 워드 벡터화
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_test_total2)
X_test_total2 = tokenizer.texts_to_sequences(X_test_total2)

# 패딩
X_test_total2 = pad_sequences(X_test_total2, maxlen = max_len)

# 라벨데이터 포함
y_test_total2 = np.array(total_data2['label'])

In [35]:
# 모델 성능 평가 
loaded_model = load_model('10_CNN_1D_KoNLPy(Mecab)_Total((movie&shopping&game&target10X)_v0.1.h5')
result = loaded_model.evaluate(X_test_total2, y_test_total2, batch_size=100)
print("\n dropout_prob1 = (0.8, 0.5) 테스트 결과:",result)

107/107 [==============================] - 1s 6ms/step - loss: 0.8407 - acc: 0.5603 - auc: 0.4914 - auc_1: 0.6821 - true_positives: 4720.0000 - true_negatives: 1234.0000 - false_positives: 1937.0000 - false_negatives: 2736.0000

 dropout_prob1 = (0.8, 0.5) 테스트 결과: [0.8407015800476074, 0.5602710247039795, 0.4914351999759674, 0.6820700764656067, 4720.0, 1234.0, 1937.0, 2736.0]


In [38]:
# 모델 성능 평가 
loaded_model = load_model('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.2.h5')
result = loaded_model.evaluate(X_test_total2, y_test_total2, batch_size=100)
print("\n dropout_prob2 = (0.2, 0.5) 테스트 결과:",result)

107/107 [==============================] - 1s 5ms/step - loss: 0.9986 - acc: 0.5507 - auc: 0.5130 - auc_1: 0.6972 - true_positives: 4497.0000 - true_negatives: 1355.0000 - false_positives: 1816.0000 - false_negatives: 2959.0000

 dropout_prob2 = (0.2, 0.5) 테스트 결과: [0.9985904097557068, 0.5506728291511536, 0.5130380988121033, 0.6972492933273315, 4497.0, 1355.0, 1816.0, 2959.0]


In [41]:
# 모델 성능 평가 
loaded_model = load_model('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.3.h5')
result = loaded_model.evaluate(X_test_total2, y_test_total2, batch_size=100)
print("\n dropout_prob3 = (0.5, 0.2) 테스트 결과:",result)

107/107 [==============================] - 1s 5ms/step - loss: 0.9704 - acc: 0.5454 - auc: 0.5183 - auc_1: 0.7041 - true_positives: 4374.0000 - true_negatives: 1422.0000 - false_positives: 1749.0000 - false_negatives: 3082.0000

 dropout_prob3 = (0.5, 0.2) 테스트 결과: [0.9704037308692932, 0.545403242111206, 0.5183006525039673, 0.7040600180625916, 4374.0, 1422.0, 1749.0, 3082.0]


In [47]:
# 모델 성능 평가 
loaded_model = load_model('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.4.h5')
result = loaded_model.evaluate(X_test_total2, y_test_total2, batch_size=100)
print("\n dropout_prob4 = (0.0, 0.0) 테스트 결과:",result)

107/107 [==============================] - 1s 5ms/step - loss: 1.0351 - acc: 0.5266 - auc: 0.5032 - auc_1: 0.6938 - true_positives: 4157.0000 - true_negatives: 1439.0000 - false_positives: 1732.0000 - false_negatives: 3299.0000

 dropout_prob4 = (0.0, 0.0) 테스트 결과: [1.0351136922836304, 0.5265832543373108, 0.5031668543815613, 0.6937627792358398, 4157.0, 1439.0, 1732.0, 3299.0]


In [49]:
# 모델 성능 평가 
loaded_model = load_model('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.5.h5')
result = loaded_model.evaluate(X_test_total2, y_test_total2, batch_size=100)
print("\n dropout_prob5 = (0.1, 0.3) 테스트 결과:",result)

107/107 [==============================] - 1s 5ms/step - loss: 1.0273 - acc: 0.5423 - auc: 0.5083 - auc_1: 0.6931 - true_positives: 4366.0000 - true_negatives: 1397.0000 - false_positives: 1774.0000 - false_negatives: 3090.0000

 dropout_prob5 = (0.1, 0.3) 테스트 결과: [1.0273441076278687, 0.542297899723053, 0.508259117603302, 0.6931098103523254, 4366.0, 1397.0, 1774.0, 3090.0]


In [53]:
# 모델 성능 평가 
loaded_model = load_model('10_CNN_1D_KoNLPy(Mecab)_Total(movie&shopping&game&target10X)_v0.1.6.h5')
result = loaded_model.evaluate(X_test_total2, y_test_total2, batch_size=100)
print("\n dropout_prob5 = (0.1, 0.1) 테스트 결과:",result)

107/107 [==============================] - 2s 5ms/step - loss: 0.9393 - acc: 0.5232 - auc: 0.5012 - auc_1: 0.6912 - true_positives: 4147.0000 - true_negatives: 1413.0000 - false_positives: 1758.0000 - false_negatives: 3309.0000

 dropout_prob5 = (0.1, 0.1) 테스트 결과: [0.9392973780632019, 0.5231956243515015, 0.5011553764343262, 0.6911966800689697, 4147.0, 1413.0, 1758.0, 3309.0]
